In [9]:
import torchvision
import matplotlib.pyplot as plt
from tqdm import tqdm
import cv2
import numpy as np
from skimage import filters
from torchvision import transforms

classes = ('anger', 'digust', 'fear', 'happy','sadness','surprise', 'normal')

path_train = './data/train/'
path_valid = './data/valid/'

transforms_train = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
])

transforms_vaild = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
])

data_train = torchvision.datasets.ImageFolder(root=path_train, transform=transforms_train)
data_valid = torchvision.datasets.ImageFolder(root=path_valid, transform=transforms_vaild)

In [8]:
data_train[2000][1]

0

In [95]:
def feature(img,frequency):
    img = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    real, imag = filters.gabor(img_gray, frequency=frequency, theta=0/180*np.pi)
    img_mod = np.sqrt(real.astype(float)**2 + imag.astype(float)**2)
    newimg = cv2.resize(img_mod, (0,0), fx=1/4, fy =1/4, interpolation=cv2.INTER_AREA)
    tempfea = newimg.flatten()
    tmean = np.mean(tempfea)
    tstd = (np.std(tempfea)+1e-6)
    newfea1 = (tempfea - tmean) / tstd
    real, imag = filters.gabor(img_gray, frequency=frequency, theta=30/180*np.pi)
    img_mod = np.sqrt(real.astype(float)**2 + imag.astype(float)**2)
    newimg = cv2.resize(img_mod, (0,0), fx=1/4, fy =1/4, interpolation=cv2.INTER_AREA)
    tempfea = newimg.flatten()
    tmean = np.mean(tempfea)
    tstd = (np.std(tempfea)+1e-6)
    newfea2 = (tempfea - tmean) / tstd
    real, imag = filters.gabor(img_gray, frequency=frequency, theta=60/180*np.pi)
    img_mod = np.sqrt(real.astype(float)**2 + imag.astype(float)**2)
    newimg = cv2.resize(img_mod, (0,0), fx=1/4, fy =1/4, interpolation=cv2.INTER_AREA)
    tempfea = newimg.flatten()
    tmean = np.mean(tempfea)
    tstd = (np.std(tempfea)+1e-6)
    newfea3 = (tempfea - tmean) / tstd
    real, imag = filters.gabor(img_gray, frequency=frequency, theta=90/180*np.pi)
    img_mod = np.sqrt(real.astype(float)**2 + imag.astype(float)**2)
    newimg = cv2.resize(img_mod, (0,0), fx=1/4, fy =1/4, interpolation=cv2.INTER_AREA)
    tempfea = newimg.flatten()
    tmean = np.mean(tempfea)
    tstd = (np.std(tempfea)+1e-6)
    newfea4 = (tempfea - tmean) / tstd
    real, imag = filters.gabor(img_gray, frequency=frequency, theta=120/180*np.pi)
    img_mod = np.sqrt(real.astype(float)**2 + imag.astype(float)**2)
    newimg = cv2.resize(img_mod, (0,0), fx=1/4, fy =1/4, interpolation=cv2.INTER_AREA)
    tempfea = newimg.flatten()
    tmean = np.mean(tempfea)
    tstd = (np.std(tempfea)+1e-6)
    newfea5 = (tempfea - tmean) / tstd
    real, imag = filters.gabor(img_gray, frequency=frequency, theta=150/180*np.pi)
    img_mod = np.sqrt(real.astype(float)**2 + imag.astype(float)**2)
    newimg = cv2.resize(img_mod, (0,0), fx=1/4, fy =1/4, interpolation=cv2.INTER_AREA)
    tempfea = newimg.flatten()
    tmean = np.mean(tempfea)
    tstd = (np.std(tempfea)+1e-6)
    newfea6 = (tempfea - tmean) / tstd
    newfea = np.hstack([newfea1, newfea2, newfea3, newfea4, newfea5, newfea6])
    return newfea

train_data, train_label = [],[]
valid_data, valid_label = [],[]
for i in range(len(data_train)):
    img = data_train[i][0]
    train_label.append(data_train[i][1])
    train_data.append(feature(img,0.5))

for i in range(len(data_valid)):
    img = data_valid[i][0]
    valid_label.append(data_valid[i][1])
    valid_data.append(feature(img,0.5))

In [96]:
train_data[1].shape

(864,)

In [97]:
from sklearn.decomposition import PCA
pca = PCA(100)
new_train_data = pca.fit_transform(train_data)
new_valid_data = pca.fit_transform(valid_data)

In [98]:
from sklearn.ensemble import AdaBoostClassifier
clf = AdaBoostClassifier()
clf.fit(new_train_data, train_label)
prediction = clf.predict(new_valid_data)

In [99]:
total = len(prediction)
correct = sum(prediction == valid_label)
accuracy = correct / total
print(accuracy)

from sklearn.metrics import classification_report
print(classification_report(valid_label , prediction))

0.27291724714405124
              precision    recall  f1-score   support

           0       0.17      0.09      0.12       958
           1       0.08      0.05      0.06       111
           2       0.19      0.08      0.12      1024
           3       0.33      0.56      0.42      1774
           4       0.27      0.24      0.25      1247
           5       0.18      0.18      0.18       831
           6       0.28      0.28      0.28      1233

    accuracy                           0.27      7178
   macro avg       0.21      0.21      0.20      7178
weighted avg       0.25      0.27      0.25      7178

